In [1]:
import sys
sys.version

'3.9.5 (tags/v3.9.5:0a7dcbd, May  3 2021, 17:27:52) [MSC v.1928 64 bit (AMD64)]'

In [2]:
import serial

## Look for open ports

In [35]:
from serial.tools import list_ports
port_data = list_ports.comports()
for port, desc, hwid in sorted(port_data):
        print(port)
        print(desc)    

COM3
Standard Serial over Bluetooth link (COM3)
COM4
Standard Serial over Bluetooth link (COM4)
COM5
USB Serial Port (COM5)


**COM5 is the port to use**

## Connecting with pyserial

In [42]:
import serial

ser = serial.Serial(
    port='COM5',
    baudrate=9600,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS,
    timeout=2
)

# Construct the Modbus message to read a register
address = 0x01
function_code = 0x03
register_address = 0x0000
register_count = 0x0010
message = bytearray([address, function_code, (register_address >> 8) & 0xFF, register_address & 0xFF, (register_count >> 8) & 0xFF, register_count & 0xFF])
# From https://github.com/myinvent/ESP32-LoRa-RS485-Agriculture-Kit/blob/main/example_program/arduino/ESP32_LoRa_TH_EC_PH_NPK_Sensor/ESP32_LoRa_TH_EC_PH_NPK_Sensor.ino
message = (0x01, 0x03, 0x00, 0x00, 0x00, 0x07, 0x04, 0x08)

# Send the message to the device
ser.write(message)

# Read the response
response = ser.readline()
ser.close()
response

b'\x01\x03\x0e\t\xf5\x00\x00\x00\x00\x00\x00\x00\x00\x00\x96\x00\x00J\xc4'

In [21]:
?response.decode(encoding='utf-')

In [16]:
register_count >> 8

0

In [17]:
register_count

16

### Decoding using chatGPT code

In [40]:
# Extract the data from the response
data = response[3:-2]

# Convert the data to a list of integers
values = [int.from_bytes(data[i:i+2], byteorder='big', signed=False) for i in range(0, len(data), 2)]

# Convert the list of integers to a string
string_data = ''.join([chr(val) for val in values])

values

[]

In [41]:
string_data

''

### PySerial combine code
https://files.seeedstudio.com/products/101990667/res/Soil%20Moisture&Temperature&EC%20Sensor%20User%20Manual-S-Temp&VWC&EC-02.pdf

In [56]:
import serial

ser = serial.Serial(
    port='COM5',
    baudrate=9600,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS,
    timeout=5
)

# Construct the Modbus message to read a register
message = (0x01, 0x03, 0x00, 0x00, 0x00, 0x07, 0x04, 0x08)

# Send the message to the device
ser.write(message)

# Read the response
response = ser.readline()

ser.close()

# Extract the data from the response
data = response[3:-2]

# Convert the data to a list of integers
values = [int.from_bytes(data[i:i+2], byteorder='big', signed=False) for i in range(0, len(data), 2)]

# Extract values
temp = values[0] # -4000 to 8000, where 1000 = 10C
temp = temp/100 # Now in units of C

water_content = values[1] # 0-10000 corresponds to 0-100%
water_content = water_content/100 #Units of percent

elec_conduct = values[2] # 0-20000 corresponds to 0-20000us/cm

salinity = values[3] # 0-20000 corresponds to 0-20000mg/L

tds = values[4] # 0-20000 corresponds to 0-20000mg/L

epsilon = values[5] # 0-8200 corresponds to 0.00~82.00
epsilon = epsilon/100

soil_type = values[6] #0-3
                    # 0: Mineral soil
                    # 1: Sandy soil
                    # 2: Clay
                    # 3: Organic soil 

print("Temp: {}, Water %: {}".format(temp, water_content))
print("Elect Cond: {}, Salinity: {}, TDS: {}".format(elec_conduct, salinity, tds))
print("Soil type: {}".format(soil_type))

Temp: 28.35, Water %: 8.29
Elect Cond: 0, Salinity: 0, TDS: 0
Soil type: 0


## PyModbus

In [ ]:
from pymodbus.client import ModbusSerialClient

client = ModbusSerialClient(method='rtu', 
                            port='COM5', 
                            baudrate=9600, timeout=1)

# Connect to the device
client.connect()

# Read a holding register
address = 0x01
register_address = 0x0000
result = client.read_holding_registers(register_address, count=1, unit=address)

if result.isError():
    print("Error reading register")
else:
    print("Value: ", result.registers[0])

# Disconnect from the device
client.close()


# Using chirp-modbus
This is a different product but it may be similar, or the code can be reused

In [12]:
import chirp_modbus
sensor = chirp_modbus.SoilMoistureSensor(address=1, serialport='COM5')
sensor.getMoisture()

NoResponseError: No communication with the instrument (no answer)